![imagenes](logo.png)

# Redes recurrentes

En una red neuronal estándar (densa), se asume que cada dato es un vector completo y estático. Todas las características están disponibles simultáneamente y no existe noción de orden. El modelo recibe al cliente de una sola vez y produce una salida sin memoria del pasado.

Este supuesto es adecuado para datos tabulares, pero falla cuando los datos presentan:

- **orden temporal**,  
- **dependencia entre observaciones**,  
- **efectos acumulativos**,  
- **evolución en el tiempo**.

Las redes neuronales recurrentes (RNN) surgen para modelar este tipo de datos.

Una RNN está diseñada para situaciones en las que el cliente **no es un objeto fijo**, sino una **historia que se desarrolla en el tiempo**. Ya no estamos frente a un edificio de información, sino ante una **secuencia ordenada de eventos**.

Imagina que un adolescente llega a consultar al experto ligador más sabio del barrio. El adolescente no llega en blanco, sino que trae toda su historia emocional ordenada en el tiempo: cada mensaje enviado, cada "visto" sin respuesta, cada cita que salió bien (o desastrosamente mal), cada señal ambigua, cada pelea, cada reconciliación. Nada ocurre aislado, pues todo es una secuencia que se va contando paso a paso.

El experto no ve al adolescente como una foto fija. Lo ve como una historia en movimiento: "Lo que pasó en cierta ocasión cambia cómo interpreta lo de del presente, y eso condiciona lo que probablemente pase mañana". Eso es exactamente una red neuronal recurrente (RNN).

## El adolescente como serie de tiempo

Cada instante en la vida amorosa del adolescente es un estado observado dentro de una secuencia:

- $x_1$: el primer “hola” en Instagram  
- $x_2$: la primera respuesta (rápida y con muchos emojis)  
- $x_3$: el primer “ok” después de 4 horas  
- $x_4$: la cita donde todo fluyó increíble  
- ...
- $x_T$: el último mensaje que acaba de llegar (“estoy ocupado” seco)

Por sí solo, un “ok” no dice casi nada. Pero un “ok” después de una pelea + ghosting de 3 días + like en storie es una señal completamente distinta. Hagamos hincapié en esto: **El presente solo cobra sentido a través del pasado.**

Formalmente, el adolescente no es un vector aislado, sino una secuencia ordenada $$(x_1, x_2, \dots, x_T)$$

En el marco de RNN, cada cliente es un adolescente que llega al experto con una historia emocional ordenada en el tiempo. No se presenta como una fotografía aislada, sino como una sucesión de estados observados:

$$
(x_1, x_2, \dots, x_T)
$$

donde cada $x_t \in \mathbb{R}^d$ representa lo ocurrido en el instante $t$: un mensaje, una respuesta tardía, una cita, un silencio.

El orden es fundamental. Intercambiar dos instantes altera completamente el significado. El presente **solo cobra sentido a través del pasado**.



## El experto ligador como RNN

El experto escucha la historia completa de golpe. No necesita verte después, pues procesa la secuencia paso a paso. En cada instante $t$:

1. Recibe el evento nuevo $x_t$  
2. Lo combina con todo lo que ya sabe hasta ese momento  
3. Actualiza su diagnóstico interno

Ese diagnóstico interno es el **estado oculto** $h_t$. Es decir, $h_t$ es lo que se opina del evento $x_t$ + la opinión de los eventos anteriores $h_{t-1}$. Es decir, $$h_t = f(h_{t-1}, x_t)$$

## Ecuación de una RNN básica

En una RNN elemental, la actualización del estado oculto se expresa como

$$
h_t = \phi(W_x x_t + W_h h_{t-1} + b)
$$

con:

- $W_x \in \mathbb{R}^{m \times d}$,
- $W_h \in \mathbb{R}^{m \times m}$,
- $b \in \mathbb{R}^m$,
- $\phi$: función no lineal (tanh o ReLU).

Un punto esencial: **los mismos pesos se utilizan para todos los instantes**. No hay un experto distinto en cada tiempo; hay **uno solo que evoluciona**.

## Esquemas de salida

Dependiendo del problema, se consideran dos configuraciones principales:

### Many-to-one

Se utiliza únicamente el último estado oculto:

$$
y = g(h_T)
$$

El experto escucha toda la historia y emite un diagnóstico final.

### Many-to-many

Se produce una salida en cada instante:

$$
y_t = g(h_t), \quad t = 1, \dots, T
$$

El experto actualiza su opinión tras cada nuevo evento.

## Profundidad temporal

A diferencia de una CNN, donde la profundidad es espacial, en una RNN la profundidad es **temporal**.

Aunque la red tenga una sola capa recurrente, al desplegarla en el tiempo se obtiene una red profunda de profundidad \(T\):

$$
(h_0, x_1) \rightarrow h_1,\;
(h_1, x_2) \rightarrow h_2,\;
\dots,\;
(h_{T-1}, x_T) \rightarrow h_T
$$

Cada paso aplica la misma transformación, acumulando información histórica.

Durante el entrenamiento, el gradiente del error respecto a estados lejanos se propaga como

$$\frac{\partial \mathcal{L}}{\partial h_t}=\frac{\partial \mathcal{L}}{\partial h_T}\prod_{k=t+1}^{T}\frac{\partial h_k}{\partial h_{k-1}}$$

y, aproximadamente,

$$
\frac{\partial h_k}{\partial h_{k-1}} \approx W_h
$$

Si los valores propios de $W_h$ son menores que 1, el producto tiende a cero:

$$
\prod_{k=t}^{T} W_h \rightarrow 0
$$

Este fenómeno se conoce como **desvanecimiento del gradiente** (*vanishing gradient*): la información lejana se pierde.

Narrativamente: el experto comienza a olvidar los primeros eventos conforme la historia se alarga.

### El experto premium: LSTM y GRU

Los expertos de alto nivel no confían en memoria volátil, pues toman notas inteligentes.

### LSTM: memoria controlada

**El meticuloso.** La arquitectura LSTM introduce un **estado de memoria** $c_t$, además del estado oculto $h_t$.

Las ecuaciones completas son:

$$f_t = \sigma(W_f [h_{t-1}, x_t])\mbox{ forget gate }$$ $$i_t = \sigma(W_i [h_{t-1}, x_t])\mbox{ input gate }$$ $$\tilde{c}_t = \tanh(W_c [h_{t-1}, x_t])\mbox{ nuevo contenido}$$ $$c_t = f_t \odot c_{t-1} + i_t \odot \tilde{c}_t$$ $$o_t = \sigma(W_o [h_{t-1}, x_t])\mbox{ output gate}$$ $$h_t = o_t \odot \tanh(c_t)$$ 

Donde  
- Forget: “esto fue drama pasajero, bórralo”  
- Input: “esto sí importa, apúntalo”  
- Output: “usa esta parte del recuerdo para opinar ahora”

El cell state fluye casi intacto → memoria de largo plazo.

### GRU: memoria simplificada

**El premium eficiente.** La GRU fusiona estado y memoria en una sola variable:

$$z_t = \sigma(W_z [h_{t-1}, x_t])\mbox{ update}$$

$$r_t = \sigma(W_r [h_{t-1}, x_t])\mbox{ reset}$$

$$\tilde{h}_t = \tanh(W_h [r_t \odot h_{t-1}, x_t])$$

$$h_t = (1 - z_t) \odot h_{t-1} + z_t \odot \tilde{h}_t$$

Decide cuánto olvidar y cuánto actualizar, sin separar explícitamente estado y memoria.

## Si el experto es muy experimentado: capas múltiples (Deep RNN)

El experto no se conforma con una sola lectura. Cada capa recurrente es un día extra de reflexión.

- **Capa 1:** interpreta eventos crudos  
- **Capa 2:** interpreta interpretaciones  
- **Capa 3:** detecta patrones psicológicos profundos

Formalmente, en la capa $\ell$

$$h_t^{(\ell)} = RNN^{(\ell)} (h_t^{(\ell-1)}, h_{t-1}^{(\ell)})$$

Las capas inferiores capturan dependencias temporales locales; las superiores, patrones temporales más abstractos.


## Salida de la capa \(i+1\) en una red recurrente

Supongamos una red neuronal recurrente con **capas apiladas**, de manera análoga a una CNN.

### Entrada a la capa \(i+1\)

Supongamos que la capa $i$ produce como salida una **secuencia de tamaño $T$ de vectores** $$
A_i = (a_{i,1}, a_{i,2}, \dots, a_{i,T}),
\qquad a_{i,t} \in \mathbb{R}^{m_i}
$$

Esta secuencia representa la historia ya procesada por las capas anteriores. Diremos que la capa $i+1$ es **recurrente** si existe un conjunto de parámetros

$$
\Theta_{i+1} = (W_x^{(i+1)}, W_h^{(i+1)}, b^{(i+1)})
$$

donde:

- $W_x^{(i+1)} \in \mathbb{R}^{m_{i+1} \times m_i}$
- $W_h^{(i+1)} \in \mathbb{R}^{m_{i+1} \times m_{i+1}}$
- $b^{(i+1)} \in \mathbb{R}^{m_{i+1}}$

Estos parámetros se **comparten en el tiempo**.

Para cada instante $t = 1,\dots,T$, la capa $i+1$ produce un estado

$$h_{i+1,t}=
\phi\!\left(
W_x^{(i+1)} a_{i,t}
+
W_h^{(i+1)} h_{i+1,t-1}
+
b^{(i+1)}
\right)
$$

con $h_{i+1,0}$ inicializado (típicamente en cero).

Al finalizar el recorrido temporal, la salida natural de la capa $i+1$ es la secuencia

$$A_{i+1}=(h_{i+1,1}, h_{i+1,2}, \dots, h_{i+1,T})$$

donde cada elemento vive en $\mathbb{R}^{m_{i+1}}$.

La salida de la capa $i+1$ puede organizarse como un tensor

$$
A_{i+1} \in \mathbb{R}^{T \times m_{i+1}}
$$

Este es el análogo estructural del edificio en CNN:

- en CNN: base espacial $\times$ altura de filtros  
- en RNN: eje temporal $\times$ dimensión del estado oculto  

## Comparación capa a capa: CNN vs RNN

### CNN

La capa $i$ produce un edificio

$$
A_i \in \mathbb{R}^{d_1 \times d_2 \times m_i}
$$

La capa $i+1$, con $m_{i+1}$ filtros, produce

$$
A_{i+1} \in \mathbb{R}^{d_1' \times d_2' \times m_{i+1}}
$$

### RNN

La capa $i$ produce una historia

$$
A_i = (a_{i,1}, \dots, a_{i,T}),
\qquad a_{i,t} \in \mathbb{R}^{m_i}
$$

La capa $i+1$ produce **otra historia de la misma longitud temporal**

$$A_{i+1}=(h_{i+1,1}, \dots, h_{i+1,T}),\qquad h_{i+1,t} \in \mathbb{R}^{m_{i+1}}
$$


## Punto clave

**Una capa recurrente no cambia la longitud temporal de la historia.**  Cambia la representación interna en cada instante.

Así como en CNN la altura del edificio cambia con el número de filtros, en RNN la dimensión del estado oculto $m_{i+1}$ cambia de capa a capa,  
mientras que el tiempo $T$ se conserva. Y así como cada capa convolucional transforma un edificio en otro edificio más abstracto, cada capa recurrente transforma **una historia** en otra historia de la misma duración, pero con **estados internos de mayor nivel de abstracción** en cada instante.